<a href="https://colab.research.google.com/github/mixidota2/kaggle-indoor/blob/main/notebook/Indoor_001_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Overview
- Baselineを構築するためのnotebook
- とりあえずデータ読んで最低限のsubをするだけを目的とする

In [2]:
!nvidia-smi

Tue Mar 30 10:07:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import os
filename = "/root/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
os.chmod(filename, 600)

In [4]:
!kaggle datasets download -d kokitanisaka/indoorunifiedwifids
!unzip indoorunifiedwifids.zip > /dev/null

 99% 458M/463M [00:04<00:00, 116MB/s] 
100% 463M/463M [00:04<00:00, 111MB/s]


In [5]:
import os
import gc
import glob 
import copy
import pickle
import random

import pandas as pd
import numpy as np

import yaml
from tqdm import tqdm
from joblib import Parallel, delayed

import seaborn as sns
import matplotlib.pyplot as plt

import cv2

import scipy.stats as stats

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import TruncatedSVD

import torch
import torch.nn as nn 
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

tqdm.pandas(position=0, leave=True)

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [6]:
# consts
N_SPLITS = 5

SEED = 42

NUM_FEATS = 20

In [7]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
def get_timestamp():
    import time
    timestamp = ''
    for i, d in enumerate(time.localtime()):
        if i == 3:
            d += 8
        timestamp += str(d) + '-'
        if i == 4:
            break
    return timestamp[:-1]
def comp_metric(xhat, yhat, fhat, x, y, f):
    intermediate = np.sqrt((xhat-x)**2 + (yhat-y)**2) + 15 * np.abs(fhat-f)
#     intermediate = np.sqrt((xhat-x)**2 + (yhat-y)**2)
    return intermediate.sum()/xhat.shape[0]

## Preprocess

In [8]:
with open(f'train_all.pkl', 'rb') as f:
  data = pickle.load( f)
with open(f'test_all.pkl', 'rb') as f:
  test_data = pickle.load(f)

In [9]:
# count n features
BSSID_FEATS = [f'bssid_{i}' for i in range(NUM_FEATS)]
RSSI_FEATS  = [f'rssi_{i}' for i in range(NUM_FEATS)]

In [10]:
# get unique wifi bssids
wifi_bssids = []
for i in range(100):
    wifi_bssids.extend(data.iloc[:,i].values.tolist())
wifi_bssids = list(set(wifi_bssids))

wifi_bssids_size = len(wifi_bssids)
print(f'BSSID TYPES: {wifi_bssids_size}')

wifi_bssids_test = []
for i in range(100):
    wifi_bssids_test.extend(test_data.iloc[:,i].values.tolist())
wifi_bssids_test = list(set(wifi_bssids_test))

wifi_bssids_size = len(wifi_bssids_test)
print(f'BSSID TYPES: {wifi_bssids_size}')

wifi_bssids.extend(wifi_bssids_test)
wifi_bssids_size = len(wifi_bssids)

BSSID TYPES: 61206
BSSID TYPES: 33042


In [11]:
# preprocess

le = LabelEncoder()
le.fit(wifi_bssids)
le_site = LabelEncoder()
le_site.fit(data['site_id'])

ss = StandardScaler()
ss.fit(data.loc[:,RSSI_FEATS])

StandardScaler(copy=True, with_mean=True, with_std=True)

In [12]:
# apply transforms

data.loc[:,RSSI_FEATS] = ss.transform(data.loc[:,RSSI_FEATS])
for i in BSSID_FEATS:
    data.loc[:,i] = le.transform(data.loc[:,i])
    data.loc[:,i] = data.loc[:,i] + 1
    
data.loc[:, 'site_id'] = le_site.transform(data.loc[:, 'site_id'])

data.loc[:,RSSI_FEATS] = ss.transform(data.loc[:,RSSI_FEATS])

In [13]:
test_data.loc[:,RSSI_FEATS] = ss.transform(test_data.loc[:,RSSI_FEATS])
for i in BSSID_FEATS:
    test_data.loc[:,i] = le.transform(test_data.loc[:,i])
    test_data.loc[:,i] = test_data.loc[:,i] + 1
    
test_data.loc[:, 'site_id'] = le_site.transform(test_data.loc[:, 'site_id'])

test_data.loc[:,RSSI_FEATS] = ss.transform(test_data.loc[:,RSSI_FEATS])

In [14]:
site_count = len(data['site_id'].unique())
data.reset_index(drop=True, inplace=True)

In [15]:
seed_everything(SEED)

## Some EDA

In [16]:
#plt.figure(figsize=(10,3))
#max_iter = 10
#for i, (name, group) in enumerate(data.groupby("path")):
#    sns.lineplot(data=group, y=RSSI_FEATS[0], x=range(group.shape[0]))
#    if i > max_iter:
#        break
#plt.figure(figsize=(10,3))
#for i, (name, group) in enumerate(data.groupby("path")):
#    sns.lineplot(data=group, y="x", x=range(group.shape[0]))
#    if i > max_iter:
#        break
#plt.figure(figsize=(10,3))
#for i, (name, group) in enumerate(data.groupby("path")):
#    sns.lineplot(data=group, y="y", x=range(group.shape[0]))
#    if i > max_iter:
#        break

In [17]:
#tmp = data.loc[:,RSSI_FEATS]
#tmp.head(10)

## Modeling

In [18]:
class IndoorDataset(Dataset):
    def __init__(self, data, flag='TRAIN'):
        self.data = data
        self.flag = flag
    def __len__(self):
        return self.data.shape[0]
    def __getitem__(self, index):
        tmp_data = self.data.iloc[index]
        if self.flag == 'TRAIN':
            return {
                'BSSID_FEATS':tmp_data[BSSID_FEATS].values.astype(float),
                'RSSI_FEATS':tmp_data[RSSI_FEATS].values.astype(float),
                'site_id':tmp_data['site_id'].astype(int),
                'x':tmp_data['x'],
                'y':tmp_data['y'],
                'floor':tmp_data['floor'],
            }
        else:
            return {
                'BSSID_FEATS':tmp_data[BSSID_FEATS].values.astype(float),
                'RSSI_FEATS':tmp_data[RSSI_FEATS].values.astype(float),
                'site_id':tmp_data['site_id'].astype(int)
            }

In [19]:
class SimpleLSTM(nn.Module):
    def __init__(self, embedding_dim = 64, seq_len=20):
        super(SimpleLSTM, self).__init__()
        self.emb_BSSID_FEATS = nn.Embedding(wifi_bssids_size, embedding_dim)
        self.emb_site_id = nn.Embedding(site_count, 2)
        self.lstm1 = nn.LSTM(input_size=256,hidden_size=128, dropout=0.3, bidirectional=False)
        self.lstm2 = nn.LSTM(input_size=128,hidden_size=16, dropout=0.1, bidirectional=False)
        self.lr = nn.Linear(NUM_FEATS, NUM_FEATS * embedding_dim)
        self.lr1 = nn.Linear(2562, 256)
        self.lr_xy = nn.Linear(16, 2)
        self.lr_floor = nn.Linear(16, 1)
        self.batch_norm1 = nn.BatchNorm1d(NUM_FEATS)
        self.batch_norm2 = nn.BatchNorm1d(2562)
        self.batch_norm3 = nn.BatchNorm1d(1)
        self.dropout = nn.Dropout(0.3)
    def forward(self, x):
        
        x_bssid = self.emb_BSSID_FEATS(x['BSSID_FEATS'])
        x_bssid = torch.flatten(x_bssid, start_dim=-2)
        
        x_site_id = self.emb_site_id(x['site_id'])
        x_site_id = torch.flatten(x_site_id, start_dim=-1)
        x_rssi = self.batch_norm1(x['RSSI_FEATS'])
        x_rssi = self.lr(x_rssi)
        x_rssi = torch.relu(x_rssi)
        
        x = torch.cat([x_bssid, x_site_id, x_rssi], dim=-1)
        x = self.batch_norm2(x)
        x = self.dropout(x)
        x = torch.relu(self.lr1(x))

        x = x.unsqueeze(-2)
        x = self.batch_norm3(x)
        x = x.transpose(0, 1)
        x, _ = self.lstm1(x)
        x = x.transpose(0, 1)
        x = torch.relu(x)
        x = x.transpose(0, 1)
        x, _ = self.lstm2(x)
        x = x.transpose(0, 1)
        x = torch.relu(x)
        xy = self.lr_xy(x)
        floor = self.lr_floor(x)
        floor = torch.relu(floor)
        return xy.squeeze(-2), floor.squeeze(-2)

In [20]:
def evaluate(net, data_loader,  device='cuda'):
    net.to(device)
    net.eval()
    x_list = []
    y_list = []
    floor_list = []
    prexs_list = []
    preys_list = []
    prefloors_list = []
    for d in tqdm(data_loader, position=0):
        data_dict = {}
        data_dict['BSSID_FEATS'] = d['BSSID_FEATS'].to(device).long()
        data_dict['RSSI_FEATS'] = d['RSSI_FEATS'].to(device).float()
        data_dict['site_id'] = d['site_id'].to(device).long()
        x = d['x'].to(device).float()
        y = d['y'].to(device).float()
        floor = d['floor'].to(device).long()
        x_list.append(x.cpu().detach().numpy())
        y_list.append(y.cpu().detach().numpy())
        floor_list.append(floor.cpu().detach().numpy())
        xy, floor = net(data_dict)
        prexs_list.append(xy[:, 0].cpu().detach().numpy())
        preys_list.append(xy[:, 1].cpu().detach().numpy())
        prefloors_list.append(floor.squeeze().cpu().detach().numpy())
    x = np.concatenate(x_list)
    y = np.concatenate(y_list)
    floor = np.concatenate(floor_list)
    prexs = np.concatenate(prexs_list)
    preys =np.concatenate(preys_list)
    prefloors = np.concatenate(prefloors_list)
    eval_score = comp_metric(x, y, floor, prexs, preys, prefloors)
    return eval_score
def get_result(net, data_loader, device='cuda'):
    net.eval()
    net.to(device)
    prexs_list = []
    preys_list = []
    prefloors_list = []
    data_dict = {}
    for d in tqdm(data_loader, position=0):
        data_dict['BSSID_FEATS'] = d['BSSID_FEATS'].to(device).long()
        data_dict['RSSI_FEATS'] = d['RSSI_FEATS'].to(device).float()
        data_dict['site_id'] = d['site_id'].to(device).long()
        xy, floor = net(data_dict)
        prexs_list.append(xy[:, 0].cpu().detach().numpy())
        preys_list.append(xy[:, 1].cpu().detach().numpy())
        prefloors_list.append(floor.squeeze(-1).cpu().detach().numpy())
    prexs = np.concatenate(prexs_list)
    preys =np.concatenate(preys_list)
    prefloors = np.concatenate(prefloors_list)
    return prexs, preys, prefloors

## Training

In [21]:
score_df = pd.DataFrame()
oof = list()
predictions = list()

oof_x, oof_y, oof_f = np.zeros(data.shape[0]), np.zeros(data.shape[0]), np.zeros(data.shape[0])
preds_x, preds_y = 0, 0
preds_f_arr = np.zeros((test_data.shape[0], N_SPLITS))

for fold, (trn_idx, val_idx) in enumerate(StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED).split(data.loc[:, 'path'], data.loc[:, 'path'])):

    train_data = data.loc[trn_idx]
    valid_data = data.loc[val_idx]
    train_dataset = IndoorDataset(train_data)
    train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2, pin_memory=True)
    valid_dataset = IndoorDataset(valid_data)
    valid_dataloader = DataLoader(valid_dataset, batch_size=128, shuffle=True, num_workers=2, pin_memory=True)
    test_dataset = IndoorDataset(test_data, 'TEST')
    test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=2, pin_memory=True)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    net = SimpleLSTM()
    net = net.to(device)

    mse = nn.MSELoss()
    mse = mse.to(device)
    optim = torch.optim.Adam(net.parameters(), lr=5e-3)

    best_loss = 1000
    num_epochs = 15
    best_epoch = 0
    for epoch in range(num_epochs):
        net.train()
        losses = []
        pbar = tqdm(train_dataloader, position=0)
        for d in pbar:
            data_dict = {}
            data_dict['BSSID_FEATS'] = d['BSSID_FEATS'].to(device).long()
            data_dict['RSSI_FEATS'] = d['RSSI_FEATS'].to(device).float()
            data_dict['site_id'] = d['site_id'].to(device).long()
            x = d['x'].to(device).float().unsqueeze(-1)
            y = d['y'].to(device).float().unsqueeze(-1)
            floor = d['floor'].to(device).long()
            xy, floor = net(data_dict)
            label = torch.cat([x, y], dim=-1)
            loss = mse(xy, label)
            loss.backward()
            optim.step()
            optim.zero_grad()
            losses.append(loss.cpu().detach().numpy())
            pbar.set_description(f'loss:{np.mean(losses)}')
            data_dict['BSSID_FEATS'] = data_dict['BSSID_FEATS'].detach()
            data_dict['RSSI_FEATS'] = data_dict['RSSI_FEATS'].detach()
            data_dict['site_id'] = data_dict['site_id'].detach()
            del x, y, xy, floor, label, data_dict, loss, d
            gc.collect()
            torch.cuda.empty_cache()
        score = evaluate(net, valid_dataloader, device)
        if score < best_loss:
            best_loss = score
            best_epoch = epoch
            best_model = copy.deepcopy(net)
        if best_epoch + 2 < epoch:
            break
        print("*="*50)
        print(f"fold {fold} EPOCH {epoch}: mean position error {score}")
        print("*="*50)
    test_x, test_y, test_floor = get_result(best_model, test_dataloader, device)
    preds_f_arr[:,fold] = test_floor
    preds_x += test_x
    preds_y += test_y

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
  0%|          | 0/1614 [00:00<?, ?it/s]

*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=
fold 0 EPOCH 0: mean position error 122.13354976976294
*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=


  0%|          | 0/1614 [00:00<?, ?it/s]

*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=
fold 0 EPOCH 1: mean position error 105.48808150108502
*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=


  0%|          | 0/1614 [00:00<?, ?it/s]

...

  0%|          | 0/1614 [00:00<?, ?it/s]

*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=
fold 4 EPOCH 13: mean position error 31.963135911421755
*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=


  0%|          | 0/80 [00:00<?, ?it/s]

*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=
fold 4 EPOCH 14: mean position error 31.56452147766356
*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=


100%|██████████| 80/80 [00:08<00:00,  9.03it/s]


In [24]:
subm = pd.read_csv('sample_submission.csv', index_col=0)

test_x = preds_x / (fold + 1) 
test_y = preds_y /(fold + 1)
print("*+"*40)
# as it breaks in the middle of cross-validation, the score is not accurate at all.
score = comp_metric(oof_x, oof_y, oof_f, data.iloc[:, -5].to_numpy(), data.iloc[:, -4].to_numpy(), data.iloc[:, -3].to_numpy())
oof.append(score)
print(f"mean position error {score}")
print("*+"*40)
preds_f_mode = stats.mode(preds_f_arr, axis=1)
preds_f = preds_f_mode[0].astype(int).reshape(-1)
test_preds = pd.DataFrame(np.stack((preds_f, test_x, test_y))).T
test_preds.columns = subm.columns
test_preds.index = test_data["site_path_timestamp"]
test_preds["floor"] = test_preds["floor"].astype(int)
predictions.append(test_preds)

*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+
mean position error 192.2107121781046
*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+


In [25]:
all_preds = pd.concat(predictions)
all_preds = all_preds.reindex(subm.index)

In [26]:
simple_accurate_99 = pd.read_csv('submission_99.csv')
all_preds['floor'] = simple_accurate_99['floor'].values

In [27]:
all_preds.to_csv('submission.csv')

## Post Processing

In [29]:
!git clone --depth 1 https://github.com/location-competition/indoor-location-competition-20 indoor_location_competition_20 > /dev/null
!rm -rf indoor_location_competition_20/data > /dev/null

Cloning into 'indoor_location_competition_20'...
remote: Enumerating objects: 1169, done.
remote: Counting objects: 100% (1169/1169), done.
remote: Compressing objects: 100% (1131/1131), done.
remote: Total 1169 (delta 38), reused 1167 (delta 38), pack-reused 0
Receiving objects: 100% (1169/1169), 411.37 MiB | 18.03 MiB/s, done.
Resolving deltas: 100% (38/38), done.
Checking out files: 100% (1145/1145), done.


In [36]:
!pip install --upgrade kaggle
!mkdir indoor_dataset
!kaggle competitions download -c indoor-location-navigation -p indoor_dataset > /dev/null
!unzip indoor_dataset/indoor-location-navigation.zip > /dev/null

     |████████████████████████████████| 61kB 3.1MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-cp37-none-any.whl size=73053 sha256=aaab3af6a4db14168e6ee12031389216a43061c883485305a52d71a557860ed4
  Stored in directory: /root/.cache/pip/wheels/a1/6a/26/d30b7499ff85a4a4593377a87ecf55f7d08af42f0de9b60303
Successfully built kaggle
  Found existing installation: kaggle 1.5.10
    Uninstalling kaggle-1.5.10:
      Successfully uninstalled kaggle-1.5.10
100% 12.7G/12.7G [02:31<00:00, 89.7MB/s]
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
n
n
no
no

None
All
n
n
exit


In [30]:
import multiprocessing
import numpy as np
import pandas as pd
import scipy.interpolate
import scipy.sparse
from tqdm import tqdm

from indoor_location_competition_20.io_f import read_data_file
import indoor_location_competition_20.compute_f as compute_f

In [31]:
def compute_rel_positions(acce_datas, ahrs_datas):
    step_timestamps, step_indexs, step_acce_max_mins = compute_f.compute_steps(acce_datas)
    headings = compute_f.compute_headings(ahrs_datas)
    stride_lengths = compute_f.compute_stride_length(step_acce_max_mins)
    step_headings = compute_f.compute_step_heading(step_timestamps, headings)
    rel_positions = compute_f.compute_rel_positions(stride_lengths, step_headings)
    return rel_positions

def correct_path(args):
    path, path_df = args
    
    T_ref  = path_df['timestamp'].values
    xy_hat = path_df[['x', 'y']].values
    
    example = read_data_file(f'{INPUT_PATH}/test/{path}.txt')
    rel_positions = compute_rel_positions(example.acce, example.ahrs)
    if T_ref[-1] > rel_positions[-1, 0]:
        rel_positions = [np.array([[0, 0, 0]]), rel_positions, np.array([[T_ref[-1], 0, 0]])]
    else:
        rel_positions = [np.array([[0, 0, 0]]), rel_positions]
    rel_positions = np.concatenate(rel_positions)
    
    T_rel = rel_positions[:, 0]
    delta_xy_hat = np.diff(scipy.interpolate.interp1d(T_rel, np.cumsum(rel_positions[:, 1:3], axis=0), axis=0)(T_ref), axis=0)

    N = xy_hat.shape[0]
    delta_t = np.diff(T_ref)
    alpha = (8.1)**(-2) * np.ones(N)
    beta  = (0.3 + 0.3 * 1e-3 * delta_t)**(-2)
    A = scipy.sparse.spdiags(alpha, [0], N, N)
    B = scipy.sparse.spdiags( beta, [0], N-1, N-1)
    D = scipy.sparse.spdiags(np.stack([-np.ones(N), np.ones(N)]), [0, 1], N-1, N)

    Q = A + (D.T @ B @ D)
    c = (A @ xy_hat) + (D.T @ (B @ delta_xy_hat))
    xy_star = scipy.sparse.linalg.spsolve(Q, c)

    return pd.DataFrame({
        'site_path_timestamp' : path_df['site_path_timestamp'],
        'floor' : path_df['floor'],
        'x' : xy_star[:, 0],
        'y' : xy_star[:, 1],
    })

In [37]:
INPUT_PATH = './indoor_dataset'

sub = pd.read_csv('submission.csv')
tmp = sub['site_path_timestamp'].apply(lambda s : pd.Series(s.split('_')))
sub['site'] = tmp[0]
sub['path'] = tmp[1]
sub['timestamp'] = tmp[2].astype(float)

processes = multiprocessing.cpu_count()
with multiprocessing.Pool(processes=processes) as pool:
    dfs = pool.imap_unordered(correct_path, sub.groupby('path'))
    dfs = tqdm(dfs)
    dfs = list(dfs)
sub = pd.concat(dfs).sort_values('site_path_timestamp')
sub.to_csv('submission_post.csv', index=False)



0it [00:00, ?it/s]

1it [00:01,  1.98s/it]

3it [00:02,  1.41s/it]

4it [00:02,  1.24s/it]

5it [00:03,  1.07s/it]

6it [00:03,  1.21it/s]

7it [00:04,  1.27it/s]

9it [00:05,  1.48it/s]

10it [00:06,  1.50it/s]

11it [00:06,  1.45it/s]

12it [00:07,  1.60it/s]

13it [00:09,  1.18s/it]

14it [00:10,  1.11it/s]

15it [00:10,  1.26it/s]

16it [00:12,  1.06s/it]

18it [00:13,  1.03it/s]

19it [00:13,  1.39it/s]

20it [00:14,  1.50it/s]

21it [00:14,  2.00it/s]

22it [00:15,  1.32it/s]

23it [00:16,  1.66it/s]

24it [00:17,  1.06it/s]

26it [00:18,  1.34it/s]

27it [00:20,  1.18s/it]

28it [00:21,  1.08it/s]

30it [00:22,  1.10it/s]

32it [00:23,  1.41it/s]

33it [00:23,  1.68it/s]

34it [00:24,  1.44it/s]

35it [00:25,  1.38it/s]

37it [00:25,  1.70it/s]

38it [00:26,  1.77it/s]

39it [00:26,  1.92it/s]

40it [00:27,  2.25it/s]

41it [00:27,  2.32it/s]

42it [00:28,  1.72it/s]

44it [00:28,  2.07it/s]

45it [00:30,  1.07it/s]

46it [00:31,  1.44it/s]

47it [00:31,  1.29it/s]

48it [00:3

## Check Tensor Shapes

In [2]:
tmp = data.loc[:9,BSSID_FEATS]
tmp_ids = []
for i in range(20):
    tmp_ids.extend(tmp.iloc[:,i].values.tolist())
tmp_ids = len(list(set(tmp_ids)))
_emb = nn.Embedding(wifi_bssids_size, 64)
_res = _emb(torch.tensor(tmp.values.astype(float)).long())
_res = torch.flatten(_res, start_dim=-2)
#torch.tensor(tmp.values).size()

tmp2 = data.loc[:9,RSSI_FEATS]
tmp2_ids = []
for i in range(20):
    tmp2_ids.extend(tmp2.iloc[:,i].values.tolist())
tmp2_ids = len(list(set(tmp2_ids)))
lr = nn.Linear(20, 1280)
_res2 = lr(torch.tensor(tmp2.values.astype(float)).float())

tmp3 = data.loc[:9,"site_id"]
_emb2 = nn.Embedding(site_count, 2)
_res3 = _emb2(torch.tensor(tmp3.values.astype(float)).long())
_res3 = torch.flatten(_res3, start_dim=-1)

NameError: ignored

In [ ]:
_res.size(), _res2.size(), _res3.size()

In [ ]:
_all = torch.cat([_res, _res2, _res3], dim=-1)
_all = nn.Linear(2562, 256)(_all)

In [ ]:
_all.size()

In [ ]:
_un = _all.unsqueeze(-2)
_un.size()

In [ ]:
_tr = _un.transpose(0, 1)
_tr.size()

In [ ]:
_lstm1 = nn.LSTM(input_size=256,hidden_size=128, dropout=0.3, bidirectional=False)
_ls1, _ = _lstm1(_tr)
_ls1.size()

In [ ]:
_xy = _ls1.transpose(0, 1)
_xy = nn.Linear(128,2)(_xy)
_xy.size()

In [ ]:
_xy.squeeze(-2).size()